# 🔊 Translation With Lecto

<a href="https://colab.research.google.com/github/video-db/videodb-cookbook/blob/nb/translation_lecto/examples/Translation_Lecto.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Overview

## Setup
---

### 📦  Installing packages 

In [ ]:
%pip install videodb

### 🔑 API Keys

Ensure you have access to [ElevenLabs](https://openai.com), [Lecto](https://wellsaidlabs.com/features/api/) and [VideoDB](https://videodb.io) API keys. If you haven't already, sign up for API access on the respective platforms.

> Get your API key from [VideoDB Console](https://console.videodb.io). ( Free for first 50 uploads, **No credit card required** ) 🎉

In [66]:
import os

os.environ["LECTO_API_KEY"] = ""
os.environ["VIDEO_DB_API_KEY"] = ""
os.environ["ELEVEN_LABS_API_KEY"] = ""

### 🎙️  ElevenLab's Voice ID  
You will also need ElevenLab's VoiceID of a Voice that you want to use.

For this demo, we will be using [Sam Elliot's Voice](https://elevenlabs.io/app/voice-lab/share/bcf1f77ee30ba698ec2808b47188b4189a69c862c073d704fde7034608eeaec5/nRjJQMMnjuyMXVyDw9TC). ElevenLabs has a large variety of voices to choose from (browse them [here](https://elevenlabs.io/voice-library)). Once finalized, copy the Voice ID from ElevenLabs and link it here.

In [67]:
voiceover_artist_id = "zT03pEAEi0VHKciJODfn"

## Implementation
---

### 🌐 Step 1: Connect to VideoDB

In [6]:
from videodb import connect

# Connect to VideoDB using your API key
conn = connect()
coll = conn.get_collection()

### 🎥 Step 2: Upload Video

Upload your product video to VideoDB using its URL or file path.

In [7]:
video = conn.upload(url="https://www.youtube.com/watch?v=L-9iwhEHi7Q")

### 🔍 Step 3: Index Spoken words 

In [8]:
video.index_spoken_words()
transcript = video.get_transcript_text()

100%|████████████████████████████████████████████████████████████████████████████████████████████████████| 100/100 [00:12<00:00,  8.04it/s]


In [ ]:
import os
import requests
import json


headers = {
    "X-API-Key": os.getenv("LECTO_API_KEY"),
    "Content-Type": "application/json",
    "Accept": "application/json",
}

responses = []

for transcript_chunk in [
    transcript[i : i + 1000] for i in range(0, len(transcript), 1000)
]:
    print(transcript_chunk)
    data = {"texts": [transcript_chunk], "to": ["hi"], "from": "en"}
    payload = json.dumps(data)

    response = requests.post('https://api.lecto.ai/v1/translate/text', headers=headers, data=payload)
    responses.append(response.json())

In [19]:
final_script = " ".join([response["translations"][0]['translated'][0] for response in responses])

In [68]:
import requests

# Call ElevenLabs API to generate voiceover
url = f"https://api.elevenlabs.io/v1/text-to-speech/{voiceover_artist_id}"
headers = {
    "xi-api-key": os.environ.get("ELEVEN_LABS_API_KEY"),
    "Content-Type": "application/json"
}
payload = {
    "model_id": "eleven_multilingual_v2",
    "text": final_script
}
elevenlabs_res = requests.request("POST", url, json=payload, headers=headers)

audio_file = "audio.mp3"
try:
    elevenlabs_res = requests.request("POST", url, json=payload, headers=headers)
    elevenlabs_res.raise_for_status()
except Exception as e:
    print("Error in generating voiceover", e)
    print(elevenlabs_res.json())
# Save the audio file
with open(audio_file, "wb") as f:
    f.write(elevenlabs_res.content)

In [71]:
from videodb import MediaType
audio = conn.upload(file_path=audio_file, media_type=MediaType.audio)

In [77]:
from videodb import play_stream
from videodb.timeline import Timeline
from videodb.asset import  VideoAsset , AudioAsset

timeline = Timeline(conn)

video_asset = VideoAsset(asset_id=video.id)
audio_asset = AudioAsset(asset_id=audio.id, disable_other_tracks=True)


timeline.add_inline(video_asset)
timeline.add_overlay(0, asset=audio_asset)


stream_url = timeline.generate_stream()
play_stream(stream_url)



'https://console.videodb.io/player?url=https://dseetlpshk2tb.cloudfront.net/v3/published/manifests/ba0361fc-1c84-4337-8619-a24dbc220959.m3u8'